# Iris Classification with PyTorch Lightning

This notebook demonstrates how to classify the Iris dataset using a simple feed-forward neural network implemented with PyTorch Lightning. TensorBoard is used for visualizing the training progress.


In [ ]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [ ]:
# Load and preprocess data
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
# Define Lightning Module
class IrisLightningModel(pl.LightningModule):
    def __init__(self):
        super(IrisLightningModel, self).__init__()
        self.fc1 = nn.Linear(4, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

    def training_step(self, batch, batch_idx):
        X, y = batch
        outputs = self(X)
        loss = F.cross_entropy(outputs, torch.argmax(y, dim=1))
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, y = batch
        outputs = self(X)
        loss = F.cross_entropy(outputs, torch.argmax(y, dim=1))
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.01)

# Initialize model
model = IrisLightningModel()


In [ ]:
# Initialize TensorBoard logger
logger = pl.loggers.TensorBoardLogger('logs', name='lightning_iris')

# Train the model
trainer = pl.Trainer(max_epochs=50, logger=logger, log_every_n_steps=1)
trainer.fit(model, train_loader, test_loader)

print("Training complete. Use TensorBoard to visualize results.")


### Visualize TensorBoard
To visualize the training logs, run the following command in your terminal:
```
tensorboard --logdir logs/lightning_iris
```
